# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

## Reference
- torch ensemble https://ensemble-pytorch.readthedocs.io/en/latest/

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
'''
!pip install --upgrade gdown

# Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

!unzip -q libriphone.zip
!ls libriphone
'''

"\n!pip install --upgrade gdown\n\n# Main link\n# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip\n!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip\n\n!unzip -q libriphone.zip\n!ls libriphone\n"

# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [2]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
import os
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, random_seed=1213):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(random_seed)
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
      print(y.shape)
      return X, y
    else:
      return X


# Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

## model 1

In [5]:
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        # TODO: apply batch normalization and dropout for strong baseline.
        # Reference: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html (batch normalization)
        #       https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html (dropout)
        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.2),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()
        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=39*nframes,
            hidden_size=256,         # rnn hidden unit
            num_layers=4,  # number of rnn layer
            batch_first=True, 
            bidirectional=True,# input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.fc = nn.Sequential(
            #nn.BatchNorm1d(120),
            BasicBlock(512, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            #nn.Dropout(0.1),
            #nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, output_dim)
        )
        self.fc2 = nn.Sequential(
            #nn.BatchNorm1d(700),
            #BasicBlock(120, hidden_dim),
            #*[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            #nn.Dropout(0.1),
            #nn.BatchNorm1d(hidden_dim),
            nn.Linear(700 ,output_dim)
        )
        self.dropout=nn.Dropout(0.1)
        
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 256, 10, 1, 1),  #[1,40,40]# [64, 128, 128]
            #nn.BatchNorm2d(1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      #[1,20,20]
            
            nn.Conv2d(256, 1, 10, 1, 1),  #[1,40,40]# [64, 128, 128]
        )

    def forward(self, x):
        #x=self.dropout(x)
        '''
        x=torch.unsqueeze(x,1)
        #print(x.size())
        x=self.cnn(x)
        
        #print(x.size())
        x=torch.squeeze(x,1)
        '''
        r_out, h_n = self.rnn(x, None)
        
        #r_out=torch.squeeze(r_out)
        r_out=r_out[:,-1,:].squeeze()
        r_out=self.fc(r_out)
        return r_out

# Hyper-parameters

In [6]:
from datetime import datetime
device = 'cuda' if torch.cuda.is_available() else 'cpu'

currentDateAndTime = str(datetime.now())
#currentDateAndTime = '2023-03-05 22:29:51.527292'
try:
    os.mkdir("model")
except:
    pass
rnn_dim=41
nframes=1
concat_nframes=nframes*rnn_dim

config = {
    'concat_nframes': concat_nframes ,    # Your seed number, you can pick your lucky number. :)
    'train_ratio': 0.9,   # Whether to use all features.
    'seed': 1213,   # validation_size = train_size * valid_ratio
    'batch_size': 512,     # Number of epochs.            
    'num_epoch': 8, 
    'learning_rate': 1e-3,              
    'model_path': './model/model_'+currentDateAndTime+'.ckpt',    # If model has not improved for this many consecutive epochs, stop training.     
    'input_dim': 39 * concat_nframes , # Your model will be saved herE
    'hidden_layers': 3,
    'hidden_dim': 256,
    'nframes':nframes
}

'''
# data prarameters
concat_nframes = 3              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213                        # random seed
batch_size = 512                # batch size
num_epoch = 10                   # the number of training epoch
learning_rate = 1e-4         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 2               # the number of hidden layers
hidden_dim = 64                # the hidden dim
'''
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = config['concat_nframes']   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = config['train_ratio']   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed =config['seed']         # random seed
batch_size = config['batch_size']        # batch size
num_epoch = config['num_epoch']      # the number of training epoch
learning_rate = config['learning_rate']      # learning rate
model_path = config['model_path']    # the path where the checkpoint will be saved


# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = config['input_dim']   # the input dim of the model, you should not change the value
hidden_layers = config['hidden_layers']    # the number of hidden layers
hidden_dim = config['hidden_dim']            # the hidden dim


# Dataloader

In [7]:
from torch.utils.data import DataLoader
import gc

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, random_seed=seed)

# get dataset
train_X=train_X.reshape(-1,rnn_dim,nframes*39)

val_X=val_X.reshape(-1,rnn_dim,nframes*39)

train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)


# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()
#dataset = torch.utils.data.ConcatDataset([train_set, val_set])
# get dataloader
train_loader= DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
dataset = torch.utils.data.ConcatDataset([train_set, val_set])

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 3086


3086it [01:02, 49.69it/s]


[INFO] train set
torch.Size([1906228, 1599])
torch.Size([1906228])
[Dataset] - # phone classes: 41, number of utterances for val: 343


343it [00:07, 44.79it/s]


[INFO] val set
torch.Size([210566, 1599])
torch.Size([210566])


In [8]:
dataset = torch.utils.data.ConcatDataset([train_set, val_set])

# Training

## Trainer3
The trainer3 implements the k-fold cross validation. I use the voting ensemble model made by Ensemble PyTorch.

In [10]:
from torchensemble import VotingClassifier
from torchensemble.utils.logging import set_logger
from torchensemble.utils import io
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold
# Define the ensemble
currentDateAndTime = datetime.now()
try:
    os.mkdir('ensemble')
except:
    pass

    
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold + 1))
    model = VotingClassifier(
        estimator= Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim),
        n_estimators=25,
        cuda=True,
    )
    
    print(datetime.now())
    

    model_path='./'

    #io.load(model, model_path)  # reload
    model_path='./ensemble/'+str(fold)
    os.mkdir(model_path)
    #print(model_path)
    #model.load_state_dict(torch.load(model_path))
    # Set the criterion
    criterion = nn.CrossEntropyLoss()
    model.set_criterion(criterion)

    # Set the optimizer
    model.set_optimizer('AdamW', lr=learning_rate, weight_decay=5e-4)

    # Train and Evaluate


    
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
    # Initialize a model, and put it on the device specified.
    model.fit(
        train_loader,
        epochs=10,
        test_loader=val_loader,
        save_dir=model_path,
    )

Fold 1
2023-03-22 13:31:44.833729
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 3.80497 | Correct: 10/512
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 2.37848 | Correct: 163/512
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 2.04859 | Correct: 213/512
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 1.91038 | Correct: 224/512
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 1.82815 | Correct: 242/512
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 1.52701 | Correct: 272/512
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.56225 | Correct: 280/512
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 1.52413 | Correct: 281/512
Estimator: 000 | Epoch: 000 | Batch: 800 | Loss: 1.29621 | Correct: 324/512
Estimator: 000 | Epoch: 000 | Batch: 900 | Loss: 1.31743 | Correct: 316/512
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 1.13345 | Correct: 345/512
Estimator: 000 | Epoch: 000 | Batch: 1100 | Loss: 1.19034 | Correct: 336/512
Estimator: 000 | Epoch: 000 | Batch: 1200 | Loss: 1.0

In [ ]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [13]:
from torchensemble import VotingClassifier
from torchensemble.utils.logging import set_logger
from torchensemble.utils import io
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import KFold
models=[]

model_pathes=['ensemble/0',
              'ensemble/1',
              'ensemble/2',
              'ensemble/3',
              'ensemble/4']
for i,path in enumerate(model_pathes):
    model = VotingClassifier(
            estimator= Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim),
            n_estimators=1,
            cuda=True,
        )
    model_path='./'+path
    io.load(model, model_path)  # reload
    models.append(model)


In [14]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:14, 59.40it/s]

[INFO] test set
torch.Size([527364, 1599])


In [15]:
import pandas as pd
def ensemble(models,features):
    
    preds=torch.zeros(len(models)+1,features.size(0),dtype=torch.int32)
    #preds[0]=df
    for i,model in enumerate(models):
        model.eval()
        with torch.no_grad():
            outputs = model(features)
            _, val_pred = torch.max(outputs, 1) 
            preds[i]=val_pred
    out=torch.zeros(features.size(0))
    for i in range(features.size(0)):
        pred=preds[:,i]
        #weight=pred[0]
        pred=torch.bincount(pred)
        #pred[weight]+=3
       
        sorted, indices = torch.sort(pred)
        '''
        if sorted.size(0)!=1 and sorted[0]==sorted[1]:
            out[i]=int(preds[0,i])
        else:
        '''
        out[i]=int(torch.argmax(pred))
        
    #print('out:',out.size())
    return out
    

In [16]:

pred = np.array([], dtype=np.int32)

with torch.no_grad():
    start=0
    end=0
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        end+=features.size(0)
        features = features.to(device).reshape(-1,rnn_dim,nframes*39)
        #print(d.size())
        start=end
        test_pred = ensemble(models,features)

        #_, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)
        #print(pred)
        
    


100%|██████████| 1031/1031 [02:50<00:00,  6.05it/s]


Make prediction.

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [17]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        y=int(y)
        f.write('{},{}\n'.format(i, y))

In [18]:
print(currentDateAndTime)

2023-03-22 13:31:44.605932
